In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [3]:
import os
import cv2
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler

import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
#from torchvision.datasets import ImageFolder

from matplotlib import pyplot as plt


device = ("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
DIR_TRAIN = "../input/autism-image-data/AutismDataset/train/"

DIR_TEST = "../input/autism-image-data/AutismDataset/test/"

In [5]:
train_imgs = []
valid_imgs = []
test_imgs = []

    
for img in os.listdir(DIR_TRAIN):
    train_imgs.append(DIR_TRAIN + img)
    
for img in os.listdir(DIR_TEST):
    test_imgs.append(DIR_TEST + img)

file = (train_imgs[0])

dataSetImgs = train_imgs+test_imgs
dataSetLen = len(dataSetImgs)

In [6]:
#%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(train_imgs[0])
imgplot = plt.imshow(img)
plt.show()


In [7]:
print("the shape of the images is",img.shape)

# class for the data

In [8]:
class AutismData(Dataset):
    
    def __init__(self, imgs_list, transforms = None):
        
        super().__init__()
        self.imgs_list = imgs_list
        #self.class_to_int = class_to_int
        self.transforms = transforms
        
        
    def __getitem__(self, index):
    
        image_path = self.imgs_list[index]
        
        #Reading image
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        #Retriving class label
        
        # labels
        label = self.imgs_list[index].split('/',5)[-1].split('.')[0]

        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
            
            
        if label == 'Autistic':
            label = 0
        
        else :
            label = 1
        
        return image, label
    def __len__(self):
        return len(self.imgs_list)
    


In [9]:
train_class = AutismData(train_imgs)


In [10]:
def get_mean_and_std(loader):
    mean = 0
    std = 0
    total_images_count  = 0
    for images , _ in loader:
        images_count_in_a_batch  = images.size(0)
        images = images.view(images_count_in_a_batch,images.size(1),-1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += images_count_in_a_batch
        
    mean /= total_images_count
    sdd /= total_images_count
    return mean, std


# Transform Variables

In [11]:
def get_mean_std(loader):
    channels_s, channels_sqr,num_batches = 0,0,0
    for data, _ in loader:
        channels_s += torch.mean(data,dim=[0,2,3])
        channels_sqr += torch.mean(data**2,dim=[0,2,3])
        num_batches += 1
        
        mean = channels_s/num_batches
        std = (channels_sqr/num_batches - mean**2)**0.5
        
        return mean, std

# Create the Train and Test Objects

In [12]:
data_set_img = AutismData(imgs_list=dataSetImgs,transforms=T.Compose([T.ToTensor(),T.Resize((224,224)),
                                                                  T.RandomRotation(10),  
                                                                #T.Normalize(mean,std)
                                                            ]))

#test_set = AutismData(imgs_list=test_imgs,transforms=T.Compose([T.ToTensor(),T.Resize((224,224))])) # TODO CREATE THE TRANSFORM FUNCTION

# Dummy DataLoaders


In [13]:
dummy_train_loader = DataLoader(dataset=data_set_img,batch_size=32)

In [14]:
mean, std = get_mean_std(dummy_train_loader)

In [15]:
print(mean)

# Data Class with Transform

In [16]:
data_set_img = AutismData(imgs_list=dataSetImgs,transforms=T.Compose([T.ToTensor(),T.Resize((224,224)),
                                                                  T.RandomRotation(10),  
                                                                #T.Normalize(mean,std)
                                                            ]))


In [17]:
print(dataSetLen)

# Splitting the Data

In [18]:
train_len ,test_len = dataSetLen - 1420, 1420


train_set , test_set = torch.utils.data.random_split(data_set_img,[train_len,test_len])


# Data Loaders

In [19]:
train_loader = DataLoader(train_set, batch_size=32,shuffle=True)

test_loader = DataLoader(test_set, batch_size=32,shuffle=True)

# Create Model Class

In [20]:
from torch.nn.modules.container import Sequential
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # first conv layers
        self.network  = nn.Sequential( 

            # TODO: make it with 6 conv layers along   with relu and max_p and use batchnorm2d
            # and three linear layers on the end and do a flatten before pass to linear layers

            #  first layer
            nn.Conv2d(3, 30, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(30),

            # second layer
            nn.Conv2d(30, 28, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) ,
            nn.BatchNorm2d(28),


            # third layer
            nn.Conv2d(28, 70, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2) ,
            nn.BatchNorm2d(70),

            # fourth layer
            nn.Conv2d(70,16, 3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(16),

        # flatten goes here: 
            nn.Flatten(),

            # fifth layer 
            nn.Linear(3136,50),
            nn.Linear(50, 2),
            nn.Softmax(dim=1)
        
            )
    
    # it's the forward function that defines the network structure
    # we're accepting only a single input in here, but if you want,
    # feel free to use more
    def forward(self, x):

        x = self.network(x)

        # in your model definition you can go full crazy and use arbitrary
        # python code to define your model structure
        # all these are perfectly legal, and will be handled correctly
        # by autograd:
        # if x.gt(0) > x.numel() / 2:
        #      ...
        #
        # you can even do a loop and reuse the same module inside it
        # modules no longer hold ephemeral state, so you can use them
        # multiple times during your forward pass
        # while x.norm(2) < 10:
        #    x = self.conv1(x)

        return x

model = Net()

In [21]:
if torch.cuda.is_available():
    model = model.cuda()

# Train the Model

In [22]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.003,momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [23]:
epochs = 10
train_loss, val_loss = [], []
accuracy_total_train, accuracy_total_val = [], []

for epoch in range(epochs):
   
    total_train_loss = 0
    total_val_loss = 0

    model.train()
    
    total = 0
    # training our model
    for idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)

        optimizer.zero_grad()

        pred = model(image)

        loss = criterion(pred, label)
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
                
    accuracy_train = total / len(train_set)
    accuracy_total_train.append(accuracy_train)

    total_train_loss = total_train_loss / (idx + 1)
    train_loss.append(total_train_loss)
    
    # validating our model
    model.eval()
    total = 0
    for idx, (image, label) in enumerate(test_loader):
        image, label = image.cuda(), label.cuda()
        pred = model(image)
        loss = criterion(pred, label)
        total_val_loss += loss.item()

        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1

    accuracy_val = total / len(test_set)
    accuracy_total_val.append(accuracy_val)

    total_val_loss = total_val_loss / (idx + 1)
    val_loss.append(total_val_loss)

   
    print("Epoch: {}/{}  ".format(epoch, epochs),
            "Training loss: {:.4f}  ".format(total_train_loss),
            "Testing loss: {:.4f}  ".format(total_val_loss),
            "Train accuracy: {:.4f}  ".format(accuracy_train),
            "Test accuracy: {:.4f}  ".format(accuracy_val))

